In [51]:
#import all modules here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [52]:
df = pd.read_csv('SPECT.train',header=None)
features = df.loc[:,df.columns!=0]
label = df[0]
features.head()

,1,2,3,4,5,6,7,8,9,10,...,13,14,15,16,17,18,19,20,21,22
0,0,0,0,1,0,0,0,1,1,0,...,1,1,0,0,0,0,0,0,0,0
1,0,0,1,1,0,0,0,1,1,0,...,1,1,0,0,0,0,0,0,0,1
2,1,0,1,0,1,0,0,1,0,1,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,0,0,0,0,0,0,0,1,0,0,...,1,0,1,1,0,0,0,0,0,0


In [53]:
#k=14
%matplotlib qt  
def plot_Elbow(features):
    sse = {}
    for no_of_cluster in range(2,50):
        kmeans = KMeans(n_clusters=no_of_cluster).fit(features)
        sse[no_of_cluster] = kmeans.inertia_
    plt.figure()
    plt.plot(list(sse.keys()), list(sse.values()))
    plt.xlabel("Number of cluster")
    plt.ylabel("SSE")
    plt.show()

In [54]:
plot_Elbow(features)

In [55]:
#s = Data Path p = Number of Cluster col = Label Column Name
def detected_indices_temp(data,p,col,model):
    class0 = data.loc[data[col]!=1,:]
    class1 = data.loc[data[col]!=0,:]
    def noise(data):
        sample = data.sample(frac = 0.4)
        for i in list(sample.index):
            data[col][i] = int(not data[col][i])
        return list(sample.index)    
    noise0 = noise(class0)
    noise1 = noise(class1)
    noisy_indices = noise0+noise1
    mislabeled = pd.concat([class0,class1])
    indices = list(mislabeled.index)

    # User Optimal Number of Cluster
    km = KMeans(n_clusters = p)
    km.fit(mislabeled.loc[:,mislabeled.columns!=col])
    label = km.labels_
    mislabeled['cluster'] = label

    #OneHotEncoding
    mislabeled = pd.get_dummies(mislabeled,columns=['cluster'])
    #RandomForestClassifier
    x_train,x_test,y_train,y_test = train_test_split(mislabeled.loc[:,mislabeled.columns!=col],mislabeled[col],test_size = 0.38)
    rf = model()
    rf.fit(x_train,y_train)
    predict = rf.predict(mislabeled.loc[:,mislabeled.columns!=col])
    mislabeled['predicted'] = predict
    detected = list()
    cnt = 0
    bad_change = 0
    for i in indices:
        if mislabeled[col][i] != mislabeled['predicted'][i]:
            detected.append(i)
    for i in detected:
        if i in noisy_indices:
            cnt+=1
        else:
            bad_change+=1
    return pd.DataFrame(data={'Total Noise':[len(noisy_indices)], 'Total Detected labels':[len(detected)], 'True Identified labels':[cnt], 'False identified labels':[bad_change], 'Total Noise Ratio':len(noisy_indices)/df.shape[0], 'True Identified Ratio':len(detected)/len(noisy_indices)})

    print(len(noisy_indices),len(detected),cnt,bad_change)        

In [58]:
#12 24
#15 32
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = detected_indices_temp(df,14,0,model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
result

,Total Noise,Total Detected labels,True Identified labels,False identified labels,Total Noise Ratio,True Identified Ratio,Model
0,32,36,16,20,0.4,1.12500,SVC
1,32,29,7,22,0.4,0.90625,Random Forest
2,32,32,19,13,0.4,1.00000,Naive Bayes
3,32,28,11,17,0.4,0.87500,Logistic Regression
4,32,20,14,6,0.4,0.62500,Decision Tree
